In [1]:
import os
import sys
import itertools
import time

import multiprocessing as mp

# Test the limits:
Using tmpfiles named in local tempfiles (one for each paralell process)
## Attention Candidate Object - tempfile
## Attention Candidate adjacency-matrix | priority-forces-network

```python
"""
Extracted from:
    https://github.com/dlanier/FlyingMachineFractal.git
    ./src/itergaters.py

Formatted for notebook viewing
"""
def write_row(complex_frame, list_tuple, par_set, row_number):
    """ write_row(complex_frame, list_tuple, par_set, row_number)
        Process one row of complex plane in the context of the list_tuple equation definition
        and write the row to the temp directory.

    Args:
        complex_frame:  z_plane complex frame dictionary of points
        list_tuple:     list of tuples data struct:   [(function_handle, ([par1, par2,... parN]) )]
        par_set:        parameters dictionary with keys:
                        'tmp_dir', 'it_max', 'max_d'
        row_number:     integer number of the row to process
    """
    if 'eq_order' in par_set:
        eq_order = par_set['eq_order']
    else:
        eq_order = 0

    if 'RANDOM_PLANE' in par_set and par_set['RANDOM_PLANE'] == True:
        # SF = np.abs(complex_frame['upper_left'] - complex_frame['bottom_right'])
        row_array = (np.random.randn(par_set['n_cols']) + np.random.randn(par_set['n_cols']) * 1j) # * SF
    else:
        left_style = np.linspace(complex_frame['upper_left'],
                                 complex_frame['bottom_left'], par_set['n_rows'])
        right_style = np.linspace(complex_frame['upper_right'],
                                  complex_frame['bottom_right'], par_set['n_rows'])
        row_array = np.linspace(left_style[row_number], right_style[row_number], par_set['n_cols'])

    it_max = par_set['it_max']
    max_d = par_set['max_d']
    Z_arr = np.zeros((3,row_array.size), complex)

    if eq_order == 0:
        col = 0
        for Z0 in row_array:
            Z_arr[0, col] = Z0
            Z_arr[1, col], Z_arr[2, col] = tuplerator(list_tuple, Z0, it_max, max_d)
            col += 1
    elif eq_order == 2:
        col = 0
        for Z0 in row_array:
            Z_arr[0, col] = Z0
            Z_arr[1, col], Z_arr[2, col] = tuplerator_2(list_tuple, Z0, it_max, max_d)
            col += 1
    elif eq_order == -1:
        col = 0
        for Z0 in row_array:
            Z_arr[0, col] = Z0
            Z_arr[1, col], Z_arr[2, col] = tuplerator_3(list_tuple, Z0, it_max, max_d, par_set)
            col += 1

    Z_arr[1, :] = Z_arr[1, :] + complex(0.0, float(row_number)) # row number as imaginary part
    # file_name = os.path.join(par_set['tmp_dir'], ahora_seq_name('row_%d_'%(row_number), '.txt'))
    file_name = os.path.join(par_set['tmp_dir'], ahora_seq_name('row_%d_' % (row_number), '.npy'))
    with open(file_name, 'wb') as file_handle:
        # Z_arr.dump(file_handle, allow_pickle=True)
        np.save(file_handle, Z_arr, allow_pickle=True)


def assemble_rows(par_set):
    """ Z0, Z, ET = assemble_rows(par_set)
    read the temporary files into the output matrices Z0, Z, and ET
    Args:
        par_set:            parameters dictionary with keys:
                            'n_rows', 'n_cols', 'tmp_dir'
    Returns:
        ET:                 Escape Time matrix (may be float - fractional escape times possible)
        Z:                  Complex matrix after iteration
        Z0:                 Complex plane matrix before iteration
    """
    n_rows = par_set['n_rows']
    n_cols = par_set['n_cols']
    tmp_dir = par_set['tmp_dir']
    
    Z0 = np.zeros((n_rows, n_cols),complex)
    Z = np.zeros((n_rows, n_cols),complex)
    ET = np.zeros((n_rows, n_cols))
    
    dir_listing = os.listdir(tmp_dir)
    for f in dir_listing:
        fnm = os.path.join(tmp_dir, f)                     # should limit to .npy files
        three_sum = np.load(fnm, allow_pickle=True)
        row_number = int(np.imag(three_sum[1, 0]))
        Z0[row_number, :] = three_sum[0, :]
        ET[row_number, :] = np.real(three_sum[1, :])
        Z[row_number, :] = three_sum[2, :]

    return Z0, Z, ET


def get_primitives(list_tuple, par_set):
    """     ET, Z, Z0 = get_primitives(list_tuple, par_set)

    Wrapper Function for all the parallel iteration functions where:
        write_row() - iterate a functional iterator over a row, write row result to temporary disk.
            tuplerator()    :iterates through the list of functions with their parameters.
            tuplerator2()   :++ passes Z_n-1 & Z_n-2 (second order Iterated Function Systerms).
            tuplerator3()   :passes expanded initial conditions and additional full parameters dict.

    Args:
        list_tuple:         [(tuple), (tuple),...], where tuple = (function_handle, parameters)
        par_set:            parameters dictionary with keys:
                                dir_path        :temporary directory (for parallel algorithm usage)
                                n_rows          :number of rows (grid of pixels over complex points)
                                n_cols          :number of columns
                                center_point    :center of figure on the complex plane
                                theta           :rotation of complex plane in the figure
                                zoom            :scaling of the complex plane in the figure
                                it_max
                                max_d

        delete_temp_dir:    (opt., default=true) delete temporary directory when finished

    Returns:
        ET:                 Escape Time matrix (may be float - fractional escape times possible)
        Z:                  Complex matrix after iteration
        Z0:                 Complex plane matrix before iteration
    """
    # check & instantiate the preliminary run parameters
    if 'delete_temp_dir' in par_set:
        delete_temp_dir = par_set['delete_temp_dir']
    else:
        delete_temp_dir = True

    par_set['tmp_dir'] = get_tmp_dir(par_set['dir_path'], 'tmp')
    complex_frame, par_set = zp.get_frame_from_dict(par_set)
    range_enumeration = np.int_(range(0, par_set['n_rows']))

    # allocate parallel pool, call multiprocessing.starmap() to write temporary row files
    n_cores = mp.cpu_count()
    core_pool = mp.Pool(processes=n_cores)
    core_pool.starmap(write_row, 
                     zip(itertools.repeat(complex_frame),
                         itertools.repeat(list_tuple),
                         itertools.repeat(par_set),
                         range_enumeration))
    core_pool.close()
    core_pool.join()

    # collect, assemble and cleanup the temporary row files
    Z0, Z, ET = assemble_rows(par_set)
    if delete_temp_dir: remove_tmp_dir(par_set['tmp_dir'])

    return ET, Z, Z0

```